# 基于记忆的协同过滤

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

headers = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('C:/Users/xh/Desktop/practice/recommend/ml-100k/ml-100k/u.data',sep='\t',names=headers)
print(df.head())
print(df.shape)
n_user = df.user_id.unique().shape[0]
n_item = df.item_id.unique().shape[0]
print(n_user) #用户的人数
print(n_item) #电影的数量

#将数据分成训练集和测试集
data_train,data_test = train_test_split(df,test_size=0.25)
print(data_train.shape)

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
(100000, 4)
943
1682
(75000, 4)


协同过滤算法基于用户群已知的行为或者意见预测当前用户可能对那些感兴趣或者喜欢什么东西，主要有基于用户的近邻推荐和基于物品的近邻推荐。

In [20]:
#插播一个dataframe的小应用
df1 = pd.DataFrame({'col1':[1,2],'col2':[3,4]},index=['a','b'])
print(df1)
for line in df1.itertuples():
    print(line)

   col1  col2
a     1     3
b     2     4
Pandas(Index='a', col1=1, col2=3)
Pandas(Index='b', col1=2, col2=4)


In [36]:
#创建两个user-item矩阵
train_data_matrix = np.zeros((n_user,n_item))
for line in data_train.itertuples():
    train_data_matrix[line[1]-1,line[2]-1] = line[3]
print(train_data_matrix)

test_data_matrix = np.zeros((n_user,n_item))
for line in data_test.itertuples():
    test_data_matrix[line[1]-1,line[2]-1] = line[3]
print(test_data_matrix)

[[5. 0. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
[[0. 3. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [39]:
#使用sklearn的pairwise_distances函数来计算余弦相似性
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix,metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T,metric='cosine')
print(user_similarity)

[[0.         0.86509626 0.97821798 ... 0.89637033 0.87666829 0.7118622 ]
 [0.86509626 0.         0.94011952 ... 0.9032444  0.904754   0.91560008]
 [0.97821798 0.94011952 0.         ... 0.89536564 0.85516724 0.97971721]
 ...
 [0.89637033 0.9032444  0.89536564 ... 0.         0.94101158 0.89996558]
 [0.87666829 0.904754   0.85516724 ... 0.94101158 0.         0.86778768]
 [0.7118622  0.91560008 0.97971721 ... 0.89996558 0.86778768 0.        ]]


In [52]:
#插播一个关于numpy的小技巧
#np.newaxis 为np.array增加一个轴
a = np.array([1,2,3,4])
print(a)
print(type(a))
print(a.shape)
b = a[:,np.newaxis]
print(b)
print(b.shape)
c = train_data_matrix[0:5,:]
print(c)
d = c.mean(axis=0)
print(d)
e= c.mean(axis=1)
print(e)

[1 2 3 4]
<class 'numpy.ndarray'>
(4,)
[[1]
 [2]
 [3]
 [4]]
(4, 1)
[[5. 0. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]]
[1.8 0.6 0.8 ... 0.  0.  0. ]
[0.44946492 0.08799049 0.0665874  0.04934602 0.23186683]


In [57]:
#进行预测
def predict(ratings,similarity,type='user'):
    if type == 'user':
        mean_user_ratings = ratings.mean(axis=1)
        rating_diff = (ratings - mean_user_ratings[:,np.newaxis])
        pred = mean_user_ratings[:,np.newaxis]+similarity.dot(rating_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
    elif type =='item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])  
    return pred

user_pred = predict(train_data_matrix,user_similarity,type='user')
item_pred = predict(train_data_matrix,item_similarity,type='item')
print(item_pred)
# print(user_pred)
        

[[0.37428744 0.39323392 0.40154356 ... 0.45273492 0.44260734 0.4497323 ]
 [0.07285509 0.08603074 0.08163388 ... 0.08690435 0.08796365 0.08804283]
 [0.06417031 0.06685116 0.06514708 ... 0.0634154  0.06601075 0.06662701]
 ...
 [0.03523288 0.04075903 0.03985707 ... 0.04623236 0.04448374 0.04580607]
 [0.12793952 0.13734876 0.14552423 ... 0.15280284 0.15058598 0.15288519]
 [0.21235346 0.20660323 0.22897844 ... 0.26478531 0.25522284 0.26234384]]


In [58]:
#进行评估，有许多评价指标，但其中最受欢迎的用来度量预测评分的准确性的指标是均方根误差 (RMSE)。
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    print(prediction)
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print('User-based CF RMSE: ' + str(rmse(user_pred, test_data_matrix))) 
print ('Item-based CF RMSE: ' + str(rmse(item_pred, test_data_matrix)))


[0.58701973 0.7854831  0.50920578 ... 0.19898768 0.39187477 0.18963852]
User-based CF RMSE: 3.124426377807664
[0.39323392 0.37936314 0.40813701 ... 0.24179455 0.22910732 0.23097784]
Item-based CF RMSE: 3.451103857051907


基于协同过滤算法面临着高稀疏度，增加了计算的复杂的，同时也存在冷启动的问题。

# 基于模型的协同过滤

基于模型的协同过滤是基于矩阵的分解（MF),主要是作为潜变量分解和降维的一个无监督的学习方法。MF的目标是从已知的评分中学习用户的潜在喜好和产品的潜在属性，随后通过用户和产品的潜在特征点击预测未来的评分。   
一个总所周知的矩阵分解方法是奇异值分解（SVD），通过使用奇异值分解，协同过滤可以被一个近似的矩阵X所制定。
一般的方程可以表示为：
      X = U S V^T
给定m x n矩阵X：
U是一个(m x r)正交矩阵
S是一个对角线上为非负实数的(r x r)对角矩阵
V^T是一个(r x n)正交矩阵
S的对角线上的元素被称为X的奇异值。

In [60]:
from scipy.sparse.linalg import svds
#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix))) 

[1.77886524 2.53191209 0.37160699 ... 0.61575717 0.09898897 0.86870617]
User-based CF MSE: 2.7151014349322584
